# Analise de Dados de Vendas
## Exemplos de Data Science com Python

Este notebook demonstra diferentes formas de analisar e visualizar dados de vendas usando bibliotecas populares de Data Science.

---
## 1. Importacao de Bibliotecas

In [ ]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuracoes de visualizacao
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_palette('husl')

# Exibir todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

print('Bibliotecas carregadas com sucesso!')

---
## 2. Carregamento e Exploracao Inicial dos Dados

In [ ]:
# Carregar o dataset
df = pd.read_csv('vendas_exemplo.csv')

# Converter coluna de data
df['data'] = pd.to_datetime(df['data'])

# Criar colunas calculadas
df['valor_bruto'] = df['quantidade'] * df['preco_unitario']
df['valor_desconto'] = df['valor_bruto'] * df['desconto']
df['valor_liquido'] = df['valor_bruto'] - df['valor_desconto']
df['mes'] = df['data'].dt.month_name()
df['dia_semana'] = df['data'].dt.day_name()

print(f'Dataset carregado: {len(df)} registros')

In [ ]:
# Visualizar primeiras linhas
df.head(10)

In [ ]:
# Informacoes sobre o dataset
print('INFORMACOES DO DATASET')
print('=' * 50)
df.info()

In [ ]:
# Verificar valores nulos
print('VALORES NULOS POR COLUNA')
print('=' * 50)
print(df.isnull().sum())

---
## 3. Estatisticas Descritivas

In [ ]:
# Estatisticas das variaveis numericas
df.describe()

In [ ]:
# Resumo geral de vendas
print('RESUMO GERAL DE VENDAS')
print('=' * 50)
print(f"Periodo: {df['data'].min().strftime('%d/%m/%Y')} a {df['data'].max().strftime('%d/%m/%Y')}")
print(f"Total de vendas: {len(df)}")
print(f"Total de itens vendidos: {df['quantidade'].sum():,}")
print(f"Faturamento bruto: R$ {df['valor_bruto'].sum():,.2f}")
print(f"Total em descontos: R$ {df['valor_desconto'].sum():,.2f}")
print(f"Faturamento liquido: R$ {df['valor_liquido'].sum():,.2f}")
print(f"Ticket medio: R$ {df['valor_liquido'].mean():,.2f}")

In [ ]:
# Contagem de valores unicos
print('VALORES UNICOS POR CATEGORIA')
print('=' * 50)
print(f"Vendedores: {df['vendedor'].nunique()}")
print(f"Regioes: {df['regiao'].nunique()}")
print(f"Produtos: {df['produto'].nunique()}")
print(f"Categorias: {df['categoria'].nunique()}")
print(f"Clientes: {df['cliente'].nunique()}")
print(f"Formas de pagamento: {df['forma_pagamento'].nunique()}")

---
## 4. Analise por Vendedor

In [ ]:
# Tabela de vendas por vendedor
vendas_vendedor = df.groupby('vendedor').agg({
    'id_venda': 'count',
    'quantidade': 'sum',
    'valor_bruto': 'sum',
    'valor_liquido': 'sum',
    'desconto': 'mean'
}).round(2)

vendas_vendedor.columns = ['Qtd Vendas', 'Itens Vendidos', 'Valor Bruto', 'Valor Liquido', 'Desconto Medio']
vendas_vendedor = vendas_vendedor.sort_values('Valor Liquido', ascending=False)
vendas_vendedor

In [ ]:
# Grafico de barras - Faturamento por vendedor
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Faturamento
cores = sns.color_palette('husl', n_colors=len(vendas_vendedor))
axes[0].bar(vendas_vendedor.index, vendas_vendedor['Valor Liquido'], color=cores)
axes[0].set_title('Faturamento Liquido por Vendedor', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Vendedor')
axes[0].set_ylabel('Valor (R$)')
axes[0].tick_params(axis='x', rotation=45)

# Adicionar valores nas barras
for i, v in enumerate(vendas_vendedor['Valor Liquido']):
    axes[0].text(i, v + 1000, f'R$ {v:,.0f}', ha='center', fontsize=10)

# Quantidade de vendas
axes[1].bar(vendas_vendedor.index, vendas_vendedor['Qtd Vendas'], color=cores)
axes[1].set_title('Quantidade de Vendas por Vendedor', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Vendedor')
axes[1].set_ylabel('Numero de Vendas')
axes[1].tick_params(axis='x', rotation=45)

for i, v in enumerate(vendas_vendedor['Qtd Vendas']):
    axes[1].text(i, v + 0.3, str(v), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 5. Analise por Regiao

In [ ]:
# Tabela de vendas por regiao
vendas_regiao = df.groupby('regiao').agg({
    'id_venda': 'count',
    'valor_liquido': 'sum',
    'quantidade': 'sum'
}).round(2)

vendas_regiao.columns = ['Qtd Vendas', 'Faturamento', 'Itens']
vendas_regiao['Participacao %'] = (vendas_regiao['Faturamento'] / vendas_regiao['Faturamento'].sum() * 100).round(1)
vendas_regiao = vendas_regiao.sort_values('Faturamento', ascending=False)
vendas_regiao

In [ ]:
# Grafico de pizza - Participacao por regiao
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pizza de faturamento
cores = sns.color_palette('husl', n_colors=len(vendas_regiao))
explode = [0.05] * len(vendas_regiao)

axes[0].pie(vendas_regiao['Faturamento'], labels=vendas_regiao.index, autopct='%1.1f%%',
            colors=cores, explode=explode, shadow=True, startangle=90)
axes[0].set_title('Participacao no Faturamento por Regiao', fontsize=14, fontweight='bold')

# Grafico de barras horizontal
axes[1].barh(vendas_regiao.index, vendas_regiao['Faturamento'], color=cores)
axes[1].set_title('Faturamento por Regiao', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Valor (R$)')

for i, v in enumerate(vendas_regiao['Faturamento']):
    axes[1].text(v + 500, i, f'R$ {v:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 6. Analise por Categoria de Produto

In [ ]:
# Vendas por categoria
vendas_categoria = df.groupby('categoria').agg({
    'id_venda': 'count',
    'quantidade': 'sum',
    'valor_liquido': 'sum',
    'preco_unitario': 'mean'
}).round(2)

vendas_categoria.columns = ['Qtd Vendas', 'Itens Vendidos', 'Faturamento', 'Preco Medio']
vendas_categoria = vendas_categoria.sort_values('Faturamento', ascending=False)
vendas_categoria

In [ ]:
# Grafico de barras empilhadas - Categoria
fig, ax = plt.subplots(figsize=(12, 6))

cores = sns.color_palette('husl', n_colors=len(vendas_categoria))
barras = ax.bar(vendas_categoria.index, vendas_categoria['Faturamento'], color=cores)

ax.set_title('Faturamento por Categoria de Produto', fontsize=14, fontweight='bold')
ax.set_xlabel('Categoria')
ax.set_ylabel('Faturamento (R$)')
ax.tick_params(axis='x', rotation=45)

# Adicionar valores
for barra in barras:
    altura = barra.get_height()
    ax.text(barra.get_x() + barra.get_width()/2., altura,
            f'R$ {altura:,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

---
## 7. Analise Temporal (Evolucao ao Longo do Tempo)

In [ ]:
# Vendas por mes
df['ano_mes'] = df['data'].dt.to_period('M')
vendas_mes = df.groupby('ano_mes').agg({
    'id_venda': 'count',
    'valor_liquido': 'sum',
    'quantidade': 'sum'
}).reset_index()

vendas_mes.columns = ['Mes', 'Qtd Vendas', 'Faturamento', 'Itens']
vendas_mes['Mes'] = vendas_mes['Mes'].astype(str)
vendas_mes

In [ ]:
# Grafico de linha - Evolucao temporal
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Faturamento mensal
axes[0].plot(vendas_mes['Mes'], vendas_mes['Faturamento'], marker='o', linewidth=2,
             markersize=10, color='#2ecc71')
axes[0].fill_between(vendas_mes['Mes'], vendas_mes['Faturamento'], alpha=0.3, color='#2ecc71')
axes[0].set_title('Evolucao do Faturamento Mensal', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Faturamento (R$)')
axes[0].grid(True, alpha=0.3)

for i, v in enumerate(vendas_mes['Faturamento']):
    axes[0].text(i, v + 2000, f'R$ {v:,.0f}', ha='center', fontsize=10)

# Quantidade de vendas mensal
axes[1].plot(vendas_mes['Mes'], vendas_mes['Qtd Vendas'], marker='s', linewidth=2,
             markersize=10, color='#3498db')
axes[1].fill_between(vendas_mes['Mes'], vendas_mes['Qtd Vendas'], alpha=0.3, color='#3498db')
axes[1].set_title('Evolucao da Quantidade de Vendas Mensal', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Numero de Vendas')
axes[1].grid(True, alpha=0.3)

for i, v in enumerate(vendas_mes['Qtd Vendas']):
    axes[1].text(i, v + 0.5, str(v), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 8. Analise por Forma de Pagamento

In [ ]:
# Vendas por forma de pagamento
vendas_pagamento = df.groupby('forma_pagamento').agg({
    'id_venda': 'count',
    'valor_liquido': 'sum'
}).round(2)

vendas_pagamento.columns = ['Qtd Vendas', 'Faturamento']
vendas_pagamento['Ticket Medio'] = (vendas_pagamento['Faturamento'] / vendas_pagamento['Qtd Vendas']).round(2)
vendas_pagamento = vendas_pagamento.sort_values('Faturamento', ascending=False)
vendas_pagamento

In [ ]:
# Grafico de rosca (donut) - Formas de pagamento
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

cores = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

# Grafico de rosca - Quantidade
wedges, texts, autotexts = axes[0].pie(vendas_pagamento['Qtd Vendas'], 
                                        labels=vendas_pagamento.index,
                                        autopct='%1.1f%%', colors=cores,
                                        wedgeprops=dict(width=0.6))
axes[0].set_title('Distribuicao por Forma de Pagamento\n(Quantidade)', fontsize=14, fontweight='bold')

# Grafico de rosca - Faturamento
wedges, texts, autotexts = axes[1].pie(vendas_pagamento['Faturamento'], 
                                        labels=vendas_pagamento.index,
                                        autopct='%1.1f%%', colors=cores,
                                        wedgeprops=dict(width=0.6))
axes[1].set_title('Distribuicao por Forma de Pagamento\n(Faturamento)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 9. Top 10 Produtos Mais Vendidos

In [ ]:
# Top produtos por faturamento
top_produtos = df.groupby('produto').agg({
    'id_venda': 'count',
    'quantidade': 'sum',
    'valor_liquido': 'sum'
}).round(2)

top_produtos.columns = ['Qtd Vendas', 'Unidades', 'Faturamento']
top_produtos = top_produtos.sort_values('Faturamento', ascending=False).head(10)
top_produtos

In [ ]:
# Grafico de barras horizontais - Top produtos
fig, ax = plt.subplots(figsize=(12, 8))

cores = sns.color_palette('viridis', n_colors=len(top_produtos))
y_pos = range(len(top_produtos))

barras = ax.barh(y_pos, top_produtos['Faturamento'], color=cores)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_produtos.index)
ax.invert_yaxis()  # Maior valor no topo
ax.set_title('Top 10 Produtos por Faturamento', fontsize=14, fontweight='bold')
ax.set_xlabel('Faturamento (R$)')

# Adicionar valores
for i, v in enumerate(top_produtos['Faturamento']):
    ax.text(v + 500, i, f'R$ {v:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 10. Analise de Descontos

In [ ]:
# Criar faixas de desconto
df['faixa_desconto'] = pd.cut(df['desconto'], 
                              bins=[-0.01, 0, 0.05, 0.10, 0.15, 1],
                              labels=['Sem desconto', 'Ate 5%', '5% a 10%', '10% a 15%', 'Acima 15%'])

# Analise por faixa
analise_desconto = df.groupby('faixa_desconto').agg({
    'id_venda': 'count',
    'valor_bruto': 'sum',
    'valor_desconto': 'sum',
    'valor_liquido': 'sum'
}).round(2)

analise_desconto.columns = ['Qtd Vendas', 'Valor Bruto', 'Desconto Dado', 'Valor Liquido']
analise_desconto

In [ ]:
# Grafico de barras agrupadas - Desconto
fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(analise_desconto))
largura = 0.35

barras1 = ax.bar([i - largura/2 for i in x], analise_desconto['Valor Bruto'], 
                 largura, label='Valor Bruto', color='#3498db')
barras2 = ax.bar([i + largura/2 for i in x], analise_desconto['Valor Liquido'], 
                 largura, label='Valor Liquido', color='#2ecc71')

ax.set_title('Comparacao Valor Bruto vs Liquido por Faixa de Desconto', fontsize=14, fontweight='bold')
ax.set_xlabel('Faixa de Desconto')
ax.set_ylabel('Valor (R$)')
ax.set_xticks(x)
ax.set_xticklabels(analise_desconto.index, rotation=45)
ax.legend()

plt.tight_layout()
plt.show()

---
## 11. Heatmap - Vendedor x Regiao

In [ ]:
# Tabela cruzada vendedor x regiao
tabela_cruzada = pd.crosstab(df['vendedor'], df['regiao'], 
                             values=df['valor_liquido'], aggfunc='sum')
tabela_cruzada = tabela_cruzada.fillna(0)
tabela_cruzada

In [ ]:
# Heatmap
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(tabela_cruzada, annot=True, fmt=',.0f', cmap='YlGnBu',
            linewidths=0.5, ax=ax, cbar_kws={'label': 'Faturamento (R$)'})

ax.set_title('Faturamento por Vendedor e Regiao', fontsize=14, fontweight='bold')
ax.set_xlabel('Regiao')
ax.set_ylabel('Vendedor')

plt.tight_layout()
plt.show()

---
## 12. Heatmap - Categoria x Forma de Pagamento

In [ ]:
# Tabela cruzada categoria x pagamento
tabela_cat_pag = pd.crosstab(df['categoria'], df['forma_pagamento'], 
                              values=df['id_venda'], aggfunc='count')
tabela_cat_pag = tabela_cat_pag.fillna(0).astype(int)

# Heatmap
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(tabela_cat_pag, annot=True, fmt='d', cmap='RdYlGn',
            linewidths=0.5, ax=ax, cbar_kws={'label': 'Qtd Vendas'})

ax.set_title('Quantidade de Vendas por Categoria e Forma de Pagamento', fontsize=14, fontweight='bold')
ax.set_xlabel('Forma de Pagamento')
ax.set_ylabel('Categoria')

plt.tight_layout()
plt.show()

---
## 13. Boxplot - Distribuicao de Valores

In [ ]:
# Boxplot por categoria
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Por categoria
sns.boxplot(data=df, x='categoria', y='valor_liquido', ax=axes[0], palette='husl')
axes[0].set_title('Distribuicao do Valor por Categoria', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Categoria')
axes[0].set_ylabel('Valor Liquido (R$)')
axes[0].tick_params(axis='x', rotation=45)

# Por vendedor
sns.boxplot(data=df, x='vendedor', y='valor_liquido', ax=axes[1], palette='husl')
axes[1].set_title('Distribuicao do Valor por Vendedor', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Vendedor')
axes[1].set_ylabel('Valor Liquido (R$)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## 14. Histograma - Distribuicao de Precos

In [ ]:
# Histograma de precos unitarios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de preco unitario
axes[0].hist(df['preco_unitario'], bins=15, color='#3498db', edgecolor='white', alpha=0.7)
axes[0].axvline(df['preco_unitario'].mean(), color='red', linestyle='--', label=f"Media: R$ {df['preco_unitario'].mean():.2f}")
axes[0].axvline(df['preco_unitario'].median(), color='green', linestyle='--', label=f"Mediana: R$ {df['preco_unitario'].median():.2f}")
axes[0].set_title('Distribuicao de Precos Unitarios', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Preco Unitario (R$)')
axes[0].set_ylabel('Frequencia')
axes[0].legend()

# Histograma de valor liquido
axes[1].hist(df['valor_liquido'], bins=15, color='#2ecc71', edgecolor='white', alpha=0.7)
axes[1].axvline(df['valor_liquido'].mean(), color='red', linestyle='--', label=f"Media: R$ {df['valor_liquido'].mean():.2f}")
axes[1].axvline(df['valor_liquido'].median(), color='green', linestyle='--', label=f"Mediana: R$ {df['valor_liquido'].median():.2f}")
axes[1].set_title('Distribuicao de Valores das Vendas', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Valor Liquido (R$)')
axes[1].set_ylabel('Frequencia')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## 15. Scatter Plot - Relacao Quantidade x Valor

In [ ]:
# Scatter plot com cores por categoria
fig, ax = plt.subplots(figsize=(12, 7))

categorias = df['categoria'].unique()
cores = sns.color_palette('husl', n_colors=len(categorias))

for i, cat in enumerate(categorias):
    dados_cat = df[df['categoria'] == cat]
    ax.scatter(dados_cat['quantidade'], dados_cat['valor_liquido'], 
               c=[cores[i]], label=cat, s=100, alpha=0.7, edgecolors='white')

ax.set_title('Relacao entre Quantidade e Valor da Venda', fontsize=14, fontweight='bold')
ax.set_xlabel('Quantidade')
ax.set_ylabel('Valor Liquido (R$)')
ax.legend(title='Categoria', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 16. Matriz de Correlacao

In [ ]:
# Selecionar apenas colunas numericas para correlacao
colunas_numericas = ['quantidade', 'preco_unitario', 'desconto', 'valor_bruto', 'valor_desconto', 'valor_liquido']
correlacao = df[colunas_numericas].corr()

# Heatmap de correlacao
fig, ax = plt.subplots(figsize=(10, 8))

mascara = np.triu(np.ones_like(correlacao, dtype=bool))
sns.heatmap(correlacao, mask=mascara, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, linewidths=0.5, ax=ax, vmin=-1, vmax=1,
            cbar_kws={'label': 'Correlacao'})

ax.set_title('Matriz de Correlacao entre Variaveis Numericas', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 17. Dashboard Resumo (Multiplos Graficos)

In [ ]:
# Dashboard com multiplos graficos
fig = plt.figure(figsize=(16, 12))

# 1. Faturamento por mes (linha)
ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(vendas_mes['Mes'], vendas_mes['Faturamento'], marker='o', color='#2ecc71', linewidth=2)
ax1.fill_between(vendas_mes['Mes'], vendas_mes['Faturamento'], alpha=0.3, color='#2ecc71')
ax1.set_title('Faturamento Mensal', fontweight='bold')
ax1.set_xlabel('Mes')
ax1.tick_params(axis='x', rotation=45)

# 2. Por vendedor (barras)
ax2 = fig.add_subplot(2, 3, 2)
ax2.bar(vendas_vendedor.index, vendas_vendedor['Valor Liquido'], color=sns.color_palette('husl', 4))
ax2.set_title('Faturamento por Vendedor', fontweight='bold')
ax2.tick_params(axis='x', rotation=45)

# 3. Por regiao (pizza)
ax3 = fig.add_subplot(2, 3, 3)
ax3.pie(vendas_regiao['Faturamento'], labels=vendas_regiao.index, autopct='%1.1f%%',
        colors=sns.color_palette('husl', 4))
ax3.set_title('Participacao por Regiao', fontweight='bold')

# 4. Por categoria (barras horizontais)
ax4 = fig.add_subplot(2, 3, 4)
ax4.barh(vendas_categoria.index, vendas_categoria['Faturamento'], color=sns.color_palette('viridis', 6))
ax4.set_title('Faturamento por Categoria', fontweight='bold')
ax4.invert_yaxis()

# 5. Por forma de pagamento (rosca)
ax5 = fig.add_subplot(2, 3, 5)
ax5.pie(vendas_pagamento['Faturamento'], labels=vendas_pagamento.index, autopct='%1.1f%%',
        colors=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], wedgeprops=dict(width=0.6))
ax5.set_title('Formas de Pagamento', fontweight='bold')

# 6. Metricas principais (texto)
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')
metricas = f"""
METRICAS PRINCIPAIS
{'='*30}

Total de Vendas: {len(df)}

Faturamento Bruto:
R$ {df['valor_bruto'].sum():,.2f}

Faturamento Liquido:
R$ {df['valor_liquido'].sum():,.2f}

Ticket Medio:
R$ {df['valor_liquido'].mean():,.2f}

Total em Descontos:
R$ {df['valor_desconto'].sum():,.2f}
"""
ax6.text(0.1, 0.5, metricas, fontsize=12, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))

plt.suptitle('DASHBOARD DE VENDAS', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 18. Exportando Resultados

In [ ]:
# Criar um relatorio resumido em Excel (opcional)
# Descomente as linhas abaixo se quiser exportar

# with pd.ExcelWriter('relatorio_vendas.xlsx', engine='openpyxl') as writer:
#     df.to_excel(writer, sheet_name='Dados Completos', index=False)
#     vendas_vendedor.to_excel(writer, sheet_name='Por Vendedor')
#     vendas_regiao.to_excel(writer, sheet_name='Por Regiao')
#     vendas_categoria.to_excel(writer, sheet_name='Por Categoria')
#     top_produtos.to_excel(writer, sheet_name='Top Produtos')
# print('Relatorio exportado com sucesso!')

In [ ]:
# Salvar dataset processado
# df.to_csv('vendas_processado.csv', index=False)
# print('Dataset processado salvo!')